In [1]:
!mkdir -p /data/gersbachlab/Revathy/IGVF/Pipeline/scripts

In [2]:
%%writefile /data/gersbachlab/Revathy/IGVF/Pipeline/scripts/Seurat_DE.R
#!/usr/bin/env Rscript
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(patchwork))
suppressPackageStartupMessages(library("argparse"))

parser <- ArgumentParser()


parser$add_argument(p, 'feature-matrix',
                      help='folder or h5 containing feature matrix')
parser$add_argument(p, '--output-dir', default='.',
                      help='output directory for plots and tables')

args <- parser$parse_args()


data <- Read10X(args$feature_matrix)

rna <- CreateSeuratObject(counts = data, project = "Sean_Perturb_seq", min.cells = 3, min.features = 0)
rna[["percent.mt"]] <- PercentageFeatureSet(rna, pattern = "^(MT|Mt|mt)-")
rna[["percent.ribo"]] <- PercentageFeatureSet(rna, pattern = "^(RP[LS]|rp[ls])")

## Pre-filtering plot

p <- ggplot(rna@meta.data, aes(nCount_RNA, nFeature_RNA, color=percent.mt))
p <- p + geom_point(size=0.2)
p <- p + geom_hline(yintercept=0, color='red')
p <- p + geom_hline(yintercept=10, color='red')
p

## Violin plot
p <- VlnPlot(rna, features = c('percent.mt', 'percent.ribo', 'nFeature_RNA', 'nCount_RNA'), ncol = 2, pt.size = 0)
p

rna <- subset(rna, subset = (nFeature_RNA > 1 & nFeature_RNA < 2500 & percent.mt < 5 & percent.ribo < 5))

rna <- NormalizeData(rna)
rna <- FindVariableFeatures(rna, nfeatures = 2000)
rna <- ScaleData(rna)

rna <- RunPCA(rna, features = VariableFeatures(object = rna))


## Dim plot
p <- DimPlot(rna, reduction = "pca")
p


## elbow plot
p <- ElbowPlot(rna)
p


## Find clusters
rna <- FindNeighbors(rna, dims = 1:8)
rna <- FindClusters(rna, resolution = 0.5)


## find cluster markers
cluster.markers <- FindAllMarkers(rna)


rna <- RunUMAP(rna, dims = 1:8)

## Dimplot after clustering
p <- DimPlot(rna, label=TRUE)+ NoLegend()
p


DefaultAssay(rna) <- 'RNA'
all.markers <- FindAllMarkers(rna)
top5 <- all.markers %>% group_by(cluster) %>% top_n(n = 5, wt = avg_log2FC)
write.csv(all.markers, file = file.path(args$output_dir, "all_markers.csv"),quote = FALSE)


Writing /data/gersbachlab/Revathy/IGVF/Pipeline/scripts/Seurat_DE.R


In [ ]:
/data/reddylab/Alex/collab/20210526_Sean/data/sc_rna_seq/count/Targeted_GEX_Lib_1/outs/filtered_feature_bc_matrix

In [2]:
library(Seurat)

NameError: name 'library' is not defined